In [1]:
# force the model to fill in a predefined template

import inspect
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

# --- The "messy" code to be refactored ---
def get_user_initials(user_data_list):
    initials_list = []
    for user in user_data_list:
        if 'name' in user and len(user['name']) > 0:
            parts = user['name'].split(' ')
            initials = ''
            for part in parts:
                initials += part[0].upper()
            initials_list.append(initials)
    return initials_list

CODE_TO_REFACTOR = inspect.getsource(get_user_initials)

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and code context are ready.")

Setup complete. Helper functions and code context are ready.


In [2]:
prompt_unstructured_cot = [
    {
        "role": "system",
        "content": "You are a senior Python developer who refactors code to be more idiomatic and readable."
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please refactor the following Python code.
        Let's think step-by-step to plan changes and then provide the refactored code.

        ```python
        {CODE_TO_REFACTOR}
        ```
        """)
    }
]

output_unstructured = get_completion(prompt_unstructured_cot)

In [3]:
display(Markdown(output_unstructured))

To refactor the provided code, we will focus on several key improvements:

1. **Adopt List Comprehensions**: Using a list comprehension minimizes lines of code and enhances readability.
2. **Use `str.join()`**: Instead of concatenating strings with `+=`, using `str.join()` is more efficient and idiomatic.
3. **Guard Clause**: We can use a check to ensure that the `'name'` key exists and is non-empty before processing.
4. **Naming Conventions**: We'll ensure that variable names are clear and follow Python's naming conventions.

### Step-by-Step Plan:

1. Replace the for-loop that creates a list of initials with a list comprehension.
2. In the list comprehension, check for non-empty names directly in the expression.
3. Use `str.join()` to join the initials, rather than concatenating them in a loop.
4. Enhance the overall formatting for better readability.

Here's the refactored code:

```python
def get_user_initials(user_data_list):
    return [
        ''.join(part[0].upper() for part in user['name'].split())
        for user in user_data_list
        if 'name' in user and user['name']
    ]
```

### Explanation of the Changes:

- **List Comprehension**: We use a list comprehension to iterate over `user_data_list`, creating a list in a single expression.
- **`str.join()`**: We use `''.join(...)` to combine the initials into a single string efficiently.
- **Condition Check**: The condition `if 'name' in user and user['name']` ensures we only process valid names.
- **Readability**: The compact form with clear expressions makes it easier to understand what the function does at a glance.

This refactored code achieves the same functionality in a more idiomatic and Pythonic way.

In [ ]:
'''
Now, we'll solve this problem by providing a strict template. 
We will define clear sections using custom tags like [THOUGHTS] and [REFACTORED_CODE]
and instruct the model to place its response only within these sections
'''

In [4]:
response_template = dedent("""
[Reasoniong start]
Insert your step-by-step reasoning and analysis of the code here.
[Reasoning Ends]

[REFACTORED CODE START]
Place only the final, complete, and refactored Python code here.
Do not use backticks.
[REFACTORED CODE END]
""")

prompt_with_template = [
    {
        "role": "system",
        "content": "You are a senior Python developer who refactors code to be more idiomatic and readable."
    },
    {
        "role": "user",
        "content": dedent(f"""
        Please refactor the following Python code.
        Let's think step-by-step to plan changes and then provide the refactored code.

        You must structure your response following the template below.
        Do not provide any additional text or output outside of the provided tags.

        ### Template:
        {response_template}
        
        <python_code>
        {CODE_TO_REFACTOR}
        </python_code>
        """)
    }
]

templated_output = get_completion(prompt_with_template)
print(templated_output)

[Reasoning start]
The original code defines a function that extracts and capitalizes the initials from user names in a list of dictionaries. 

1. **List Comprehension**: The use of a for loop to build `initials_list` can be replaced with a list comprehension for improved readability and conciseness.
2. **String Concatenation**: Instead of accumulating initials using `initials += ...`, we can use a generator expression combined with `join` for a more efficient way of building the initials string.
3. **Conditions**: The check for 'name' in the dictionary and its length can be simplified by using the `get` method with a default value.
4. **Variable Naming**: Consider using more descriptive variable names for better clarity, but in this case, the existing names are fairly understandable.

The refactor will maintain the original functionality while making the code cleaner and more Pythonic.
[Reasoning Ends]

[REFACTORED CODE START]
def get_user_initials(user_data_list):
    return [
       